In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [2]:
df= pd.read_excel('cleandata109.xlsx', index_col= 0)

I want to choose a dataset from the filtered one ( z score etc. ) and then do these correlation thingies for the numeric data. 

In [3]:
identifier_columns = ['Country','ASBH02A','avgscore']
demographic_info_columns = ['ASBH02B', 'ASBH03A', 'ASBH04', 'ASBH15A', 'ASBH15B', 'ASBH16', 'ASBH17A', 'ASBH17B', 'ASBH18AA', 'ASBH18AB', 'ASBG01', 'ASBG03', 'ASDAGE','MINAGEARRIVAL' ]
positive_feelings_in_school = ['ASBG10A', 'ASBG10B','ASBG10C', 'ASBG10D', 'ASBG10E', 'ASBG10F']
negativeexperience_in_school_columns = ['ASBG11A', 'ASBG11B', 'ASBG11C', 'ASBG11D', 'ASBG11E', 'ASBG11F', 'ASBG11G', 'ASBG11H', 'ASBG11I', 'ASBG11J']
assessment_score_columns = ['ASRREA01', 'ASRREA02', 'ASRREA03', 'ASRREA04', 'ASRREA05', 'ASRLIT01', 'ASRLIT02', 'ASRLIT03', 'ASRLIT04', 'ASRLIT05', 'ASRINF01', 'ASRINF02', 'ASRINF03', 'ASRINF04', 'ASRINF05', 'ASRIIE01', 'ASRIIE02', 'ASRIIE03', 'ASRIIE04', 'ASRIIE05', 'ASRRSI01', 'ASRRSI02', 'ASRRSI03', 'ASRRSI04', 'ASRRSI05']
averages = ['reading_avg', 'literary_purpose_avg', 'informational_purpose_avg','interpreting_process_avg', 'straightforward_process_avg','avgscore']

In [4]:
df['avgscore'] = df[positive_feelings_in_school].mean(axis=1)

## The below is for comparing columns that are non-numeric. 

In [5]:
df['avgscore_binned'] = pd.cut(df['avgscore'], bins=10)

In [6]:
from scipy.stats import chi2_contingency

In [7]:
# Create a contingency table
contingency_table = pd.crosstab(df['ASBH02A'], df['avgscore_binned'])

In [8]:
contingency_table.head()

avgscore_binned,"(0.997, 1.3]","(1.3, 1.6]","(1.6, 1.9]","(1.9, 2.2]","(2.2, 2.5]","(2.5, 2.8]","(2.8, 3.1]","(3.1, 3.4]","(3.4, 3.7]","(3.7, 4.0]"
ASBH02A,,,,,,,,,,
No,1317,496,256,190,88,32,36,13,14,16
Yes,13843,5485,3025,2071,1115,298,361,126,110,163


In [9]:
def cramers_v(confusion_matrix):
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    r, k = confusion_matrix.shape
    return np.sqrt(chi2 / (n * (min(r, k) - 1)))

# Create a confusion matrix
confusion_matrix = pd.crosstab(df['ASBH02A'], df['avgscore_binned'])

# Calculate Cramér's V
cramers_v_value = cramers_v(confusion_matrix)
print(f"Cramér's V: {cramers_v_value}")

Cramér's V: 0.015985714584942134



Cramér's V is a statistical measure used to assess the strength of association between two nominal (categorical) variables. It is derived from the Chi-square statistic and ranges from 0 to 1, where:

0 indicates no association between the variables.
1 indicates a perfect association between the variables.
The value you provided, 0.077, is relatively close to 0, indicating a very weak association between the two variables.

Here's a general interpretation of Cramér's V:

0 to 0.1: Little or no association\
0.1 to 0.3: Weak association\
0.3 to 0.5: Moderate association\
Above 0.5: Strong association\

Since your Cramér's V is around 0.077, it suggests that the relationship between the two categorical variables in your analysis is negligible or very weak.

In [10]:
# Perform Chi-Square Test of Independence
chi2, p, dof, expected = chi2_contingency(contingency_table)

In [11]:
print(f"Chi-Square Test Statistic: {chi2}")
print(f"P-value: {p}")

Chi-Square Test Statistic: 7.424803921839236
P-value: 0.5929809854019423


1. Chi-Square Test Statistic: 245.83458534739597
The Chi-Square test statistic quantifies how much the observed frequencies in your data deviate from the expected frequencies under the assumption that the two categorical variables are independent.
A higher Chi-Square value generally indicates a greater difference between the observed and expected frequencies, suggesting a stronger association between the variables.
In your case, the Chi-Square statistic is quite large, indicating a noticeable deviation from the expected distribution if the variables were independent.
2. P-value: 7.553360938216542e-48
The P-value represents the probability of observing a Chi-Square statistic as extreme as, or more extreme than, the one calculated, under the null hypothesis (which states that the two variables are independent).
A very small P-value (like the one here, which is effectively 0.000...0000755) indicates that the observed association is highly unlikely to have occurred by chance.
Common thresholds for significance are 0.05, 0.01, and 0.001. Your P-value is far smaller than any of these, meaning you can reject the null hypothesis with extremely high confidence, suggesting a statistically significant association between the two variables.
Summary
Chi-Square Test Statistic (245.83): Indicates a substantial deviation from what would be expected under independence.
P-value (~0): Shows that this deviation is extremely unlikely to be due to random chance, indicating a statistically significant association.
However, the earlier Cramér's V value of 0.077 suggests that while the association is statistically significant, it is not strong. This can happen when the sample size is large, leading to statistically significant results even for weak associations.